# Analyze Vdbench Logs

In [ ]:
S3_LOGS_URL = ''

# Prerequsite

In [ ]:

# SET log level
# logging
import logging
logger = logging.getLogger()
logger.setLevel(logging.WARN)
logging.info("Prerequisite")

# extract credentials
import configparser
config = configparser.ConfigParser()
config.read('logs.conf')
username = config.get("data", 'user') 
password = config.get("data", 'password')


# Update fedora
!dnf update -y

# Logs operations
from benchmark_runner.jupyterlab.templates.logs_operations.logs_operations import LogsOperations
logs_operations = LogsOperations(s3_logs_url=S3_LOGS_URL)
# Logs cleanup
logs_operations.cleanup()

from benchmark_runner.jupyterlab.templates.analyze_workloads.analyze_vdbench_logs.analyze_vdbench_logs import AnalyzeVdbenchLogs
analyze_vdbench_logs = AnalyzeVdbenchLogs(s3_logs_url=S3_LOGS_URL)

## Download S3 logs

In [ ]:
logs_operations.download_s3_logs(username=username, password=password)

## Untar logs & chmod 

In [ ]:
logs_operations.untar_and_chmod_logs()

# Get vdbench log files

In [ ]:
csv_log_files = analyze_vdbench_logs.get_vdbench_csv_log_files()

# Display Dataframe

In [ ]:
analyze_vdbench_logs.display_df()

## Vdbench latency (sec)

In [ ]:
analyze_vdbench_logs.display_latency_horizontal()

## Vdbench IOPS

In [ ]:
analyze_vdbench_logs.display_iops_horizontal()